In [ ]:
import torch
from torch.nn import Module
import torch.nn as nn
import numpy as np
import pandas as pd
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:
file = 'Preprocessed data/processed_results.csv'
my_df = pd.read_csv(file)


In [ ]:
#create a model class that inherits nn.modules
class Model(nn.Module):
    #Input features(The number of object's feature)
    #First hidden layer(The number of neurons in first layer)
    #Second hidden layer(The number of neurons in second layer)
    #Output(The number of classes in which depends on objects)
    def __init__(self, in_features = 5, hidden1 = 12, hidden2 = 12, out_features = 2):
        #To instantiate our nn.Module
        super().__init__()
        #To conect all neurons with eachother
        self.fc1 = nn.Linear(in_features,hidden1)
        self.fc2 = nn.Linear(hidden1,hidden2)
        self.out = nn.Linear(hidden2,out_features)
        #A function which moves everthing forward
    def forward(self,x):
        #relu:>Rectified linear unit:>It is defined as h = max(0, a) where a is any real number.
        #If a is less than or equal to 0, the function returns 0, otherwise, it returns a
        #This function moves data from input to first hidden layer and then second and then output
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [ ]:
#Pick a manual seed for randomization
#Random number seeking
torch.manual_seed(41)
model = Model()

file = 'Preprocessed data/processed_results.csv'
my_df = pd.read_csv(file)

In [ ]:
#Train Test Split. set x,y
x = my_df.drop('Output',axis=1)
y = my_df['Output']
#Convert these numpy arrays
x = x.values
y = y.values

#Running train test split
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.3 , random_state = 45)# 70% is train and 30% is test

#Convert x features to float tensors
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)

#Convert y features to tensors long
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

#Set criterion of model to measure the error, how far off the predictions are from data
criterion = nn.CrossEntropyLoss()

#Choose an Adam optimizer , learning rate = if the error doesn't go down after a bunch of iterations(epochs), learning rate will decrease
optimizer = torch.optim.Adam(model.parameters(),lr=0.008)


In [ ]:
#Train our model
#Epoch:> one run through all the training data in network
epochs = 1900
losses = []
for i in range(epochs):

    #Go through and make a prediction
    y_pred = model.forward(x_train) #Getting the predicted results

    #Measuring the loss which will be high at first
    loss = criterion(y_pred,y_train) #Predicted values vs y_train
    losses.append(loss.detach().numpy())

    #Print every 10 epoch
    if i % 10 == 0:
        print(f"'Epoch: {i} and loss: {loss}")
    #Do back propqgation:> take the error rate of forward propagation and feed it back though the network to fine tune weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
plt.plot(range(epochs),losses)
plt.ylabel("loss/error")
plt.xlabel("epoch")

In [ ]:
#Evaluate model on test dataset(test validation)
with torch.no_grad(): #Basically turn off back propogation
    y_eval = model.forward(x_test) #x_test are features form test set and, y_eval will be our predictions
    loss = criterion(y_eval,y_test) #Find the loss or error

In [ ]:
loss

In [ ]:
#This will tell us the nomber of correct predictions for test set.
True_One = 0
False_True = 0
False_Zero = 0
True_False = 0

with torch.no_grad():
    y_pred = model(x_test)
    predicted_classes = y_pred.argmax(dim=1)
    cm = confusion_matrix(y_test, predicted_classes)
        #print(f'{i+1}.)   {str(y_val)} \t {y_test[i]} \t {y_val.argmax().item()}')

    #    if predicted_class == 1 and  true_class == 1:
           # True_One += 1

     #   elif predicted_class == 0 and true_class == 0:
          #  False_Zero += 1

      #  elif predicted_class ==1 and true_class == 0:
         #   False_True += 1

       # else:
        #    True_False += 1
'''        precision = True_One / (True_One + False_True)
        recall = True_One / (True_One + True_False)
        F1_score = (2 * precision * recall) / (precision + recall)

print(f'True:> {True_One}')
print(f'False:> {False_Zero}')
print(f'False_Pos:> {False_True}')
print(f'False_Neg:> {True_False}')
print(f'Precision:> {precision}')
print(f'Recall:> {recall}')
print(f'F1-score:> {F1_score}')'''
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()
